# Imports

In [6]:
import json
import requests

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import dataframes

## Heroes id

In [38]:
f = open('data/hero_id.json',)
data = json.load(f)['heroes']
heroes = pd.read_json(json.dumps(data))
heroes.drop(['localized_name'],axis=1,inplace=True)
heroes.set_index('id',inplace=True)

## Matches id

In [ ]:
url = 'https://pt.dotabuff.com/matches'

#game_mode=all_pick&region=south_america&skill_bracket=very_high_skill

params = dict(
    game_mode='all_pick',
    region='south_america',
    skill_bracket='very_high_skill'
)

resp = requests.get(url=url, params=params)
data = resp.json() # Check the JSON Response Content documentation below